In [1]:
def normalize(X):
    for i in range(X.shape[1]):
        X[:,i] = (X[:,i] - X[:,i].mean())/X[:,i].std()
    return X

In [2]:

# from sklearn import tree
import pandas as pd
import numpy as np
def get_data():
    """
    Returns the data from the xlsx file
    """
    file_name = 'yeast.csv' 
    df = pd.read_csv(file_name, index_col=0, header=None)
    # print(df.head()) # print the first 5 rows
    return df

k = 10
p = 8

md = get_data()
md.dropna(inplace = True)
print(md.shape)
md.replace('?', 0, inplace = True)

md.head(10)


(1484, 9)


,1,2,3,4,5,6,7,8,9
0,,,,,,,,,
ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT
AATC_YEAST,0.51,0.40,0.56,0.17,0.5,0.5,0.49,0.22,CYT
ABC1_YEAST,0.50,0.54,0.48,0.65,0.5,0.0,0.53,0.22,MIT
BAF1_YEAST,0.48,0.45,0.59,0.20,0.5,0.0,0.58,0.34,NUC
ABF2_YEAST,0.55,0.50,0.66,0.36,0.5,0.0,0.49,0.22,MIT


Convert continuous data to categorical one

In [3]:

for i in range(p):
    md[i+1] = pd.cut(md[i+1].values, 5, [0,0.2,0.4,0.6,0.8,1])
md.head(10)
# print(md[1].value_counts())


,1,2,3,4,5,6,7,8,9
0,,,,,,,,,
ADT1_YEAST,"(0.466, 0.644]","(0.478, 0.652]","(0.368, 0.526]","(-0.001, 0.2]","(0.5, 0.6]","(-0.00083, 0.166]","(0.438, 0.584]","(0.2, 0.4]",MIT
ADT2_YEAST,"(0.288, 0.466]","(0.652, 0.826]","(0.368, 0.526]","(0.2, 0.4]","(0.5, 0.6]","(-0.00083, 0.166]","(0.438, 0.584]","(0.2, 0.4]",MIT
ADT3_YEAST,"(0.466, 0.644]","(0.478, 0.652]","(0.368, 0.526]","(-0.001, 0.2]","(0.5, 0.6]","(-0.00083, 0.166]","(0.438, 0.584]","(0.2, 0.4]",MIT
AAR2_YEAST,"(0.466, 0.644]","(0.304, 0.478]","(0.526, 0.684]","(-0.001, 0.2]","(0.5, 0.6]","(-0.00083, 0.166]","(0.438, 0.584]","(0.2, 0.4]",NUC
AATM_YEAST,"(0.288, 0.466]","(0.304, 0.478]","(0.368, 0.526]","(0.4, 0.6]","(0.5, 0.6]","(-0.00083, 0.166]","(0.438, 0.584]","(0.2, 0.4]",MIT
AATC_YEAST,"(0.466, 0.644]","(0.304, 0.478]","(0.526, 0.684]","(-0.001, 0.2]","(0.5, 0.6]","(0.498, 0.664]","(0.438, 0.584]","(0.2, 0.4]",CYT
ABC1_YEAST,"(0.466, 0.644]","(0.478, 0.652]","(0.368, 0.526]","(0.6, 0.8]","(0.5, 0.6]","(-0.00083, 0.166]","(0.438, 0.584]","(0.2, 0.4]",MIT
BAF1_YEAST,"(0.466, 0.644]","(0.304, 0.478]","(0.526, 0.684]","(-0.001, 0.2]","(0.5, 0.6]","(-0.00083, 0.166]","(0.438, 0.584]","(0.2, 0.4]",NUC
ABF2_YEAST,"(0.466, 0.644]","(0.478, 0.652]","(0.526, 0.684]","(0.2, 0.4]","(0.5, 0.6]","(-0.00083, 0.166]","(0.438, 0.584]","(0.2, 0.4]",MIT


In [15]:
X = md.iloc[:, 0:8].values
outputs = ['CYT', 'NUC', 'MIT', 'ME3', 'ME2', 'ME1', 'EXC', 'VAC', 'POX', 'ERL']

# add 1 to the first column
# X = np.insert(X, 0, 1, axis=1)
# np.random.shuffle(X)

print('X:', X)
print(X.shape)

# X = normalize(X)
# print(X)

M = X.shape[0]
Y = md.iloc[:, 8:9].values.reshape(-1, 1)

y = np.zeros((len(Y), 10))
for i in range(len(Y)):
    for j in range(10):
        if outputs[j] == Y[i]:
            y[i][j] = 1
        else:
            y[i][j] = 0

X: [[0.58 0.61 0.47 ... 0.   0.48 0.22]
 [0.43 0.67 0.48 ... 0.   0.53 0.22]
 [0.64 0.62 0.49 ... 0.   0.53 0.22]
 ...
 [0.67 0.57 0.36 ... 0.   0.56 0.22]
 [0.43 0.4  0.6  ... 0.   0.53 0.39]
 [0.65 0.54 0.54 ... 0.   0.53 0.22]]
(1484, 8)


In [5]:
outputs = ['CYT', 'NUC', 'MIT', 'ME3', 'ME2', 'ME1', 'EXC', 'VAC', 'POX', 'ERL']

y = np.zeros(len(Y))

for i in range(len(Y)):
    y[i] = outputs.index(Y[i][0])


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB

y = np.zeros(len(Y))
for i in range(len(Y)):
      y[i] = np.where(outputs == Y[i])[0][0]
print(y)
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
gnb = ComplementNB()
gnb.fit(X_train, Y_train)
y_pred = gnb.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

[2. 2. 2. ... 4. 1. 0.]
Number of mislabeled points out of a total 149 points : 72
Accuracy: 0.5167785234899329


In [46]:
from sklearn.naive_bayes import GaussianNB
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
gnb = GaussianNB()
gnb.fit(X_train, Y_train)
y_pred = gnb.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Number of mislabeled points out of a total 149 points : 130
Accuracy: 0.12751677852348994
